In [1]:
import sys
from pathlib import Path
parent_dir = Path('.').resolve().parent
sys.path.insert(0, str(parent_dir))

import numpy as np
import pandas as pd
import math
import random
import statistics
import importlib

import math_expressions as mexpr
importlib.reload(mexpr)

import methods
importlib.reload(methods)

<module 'methods' from '/Users/preetkaria/Desktop/RA/time_to_leave/methods.py'>

In [2]:
df = pd.read_csv('../data/gen_data.csv')
display(df.head())

,alpha,beta,h,c,N,n,mean_n,std_n,alpha_hat,beta_hat,u,u_star,u_star_hat,z,optimal_cost,actual_cost,intervals_str
0,2,1,0.666667,25,11,5,1.873082,1.461553,2,1.140439,17.568911,14.624458,17.011105,0.859701,1.962969,0.371871,4.075370888878726_1.1961676454020844_0.9112248...
1,2,4,0.133333,10,11,5,5.342704,2.689391,4,1.353776,84.592079,65.510277,45.236862,1.448161,2.544240,5.247362,4.13826760376251_8.478514581903736_1.660076247...
2,2,4,0.133333,10,21,5,7.936178,4.567746,3,2.629011,154.519799,143.636724,142.921733,1.005003,1.451077,1.546409,11.819341605892342_13.645102878552056_2.912788...
3,3,4,0.666667,20,14,5,7.909362,3.776084,4,1.802776,168.547142,169.554063,91.491101,1.853230,20.000000,51.370694,9.435224942661229_6.3361516932619235_13.075884...
4,3,2,0.133333,30,37,5,3.944788,3.060492,2,2.374427,226.232805,185.745572,140.319334,1.323735,5.398298,11.455129,6.0115182856632785_2.589541520901462_2.0345337...


In [3]:
methods.plot_plotly(data=df['actual_cost'], data_label='Actual Cost')
methods.plot_plotly(data=df['optimal_cost'], data_label='Optimal Cost')
methods.multi_plot_plotly(data=[df['optimal_cost'], df['actual_cost']], data_label=['Optimal Cost', 'Actual Cost'])
